In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
#import random
#import sklearn
#import collections
#from sklearn.model_selection import train_test_split
#import json
#import pylab
#from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import matplotlib
import matplotlib.patches as mpatches
#import shap
#import pandas as pd
import tensorflow as tf
#import tarfile
from tensorflow.keras.models import load_model
#from qkeras import QActivation, QDense, QConv2D, QBatchNormalization
import ensembler_functions as ef
import tf2onnx
import onnx
import os
from sklearn.model_selection import train_test_split
#import load_and_match as lam

2025-03-14 20:58:38.164415: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-14 20:58:41.895779: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set matplotlib default color cycle
new_color_cycle = [
    '#1f77b4',
    '#ff7f0e',
    '#2ca02c',
    '#d62728',
    '#9467bd',
    '#8c564b',
    '#e377c2',
    '#7f7f7f',
    '#bcbd22',
    '#17becf',
    '#aec7e8',
    '#ffbb78',
    '#98df8a',
    '#ff9896',
    '#c5b0d5',
    '#c49c94',
    '#f7b6d2',
    '#c7c7c7',
    '#dbdb8d',
    '#9edae5'
]

# You can then apply this new color cycle to your matplotlib plots
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=new_color_cycle)

In [3]:
L1AD_rate = 1000
target_rate = 10

In [ ]:
# First, one should run run_lam.py to generate match L1AD scores to the events, and then set that path
# in ensembler_functions load_and_preprocess

In [5]:
# Success! These AD scores are the same.

### Main (now using the folded model)

In [2]:
training_info = {
    "save_path": "/eos/home-m/mmcohen/ad_trigger_development/trained_models/trial_136", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": False, 
    "num_trainings": 10, 
    "training_weights": True, 
    "obj_type": "HLT", 
    "overlap_removal": False,
    "duplicate_removal": False
}

data_info = {
    "train_data_scheme": "topo2A_train", 
    "pt_normalization_type": "global_division", 
    "L1AD_rate": 1000, 
    "pt_thresholds": [50, 30, 30, 30], 
    "pt_scale_factor": 0.05, 
    "comments": "back to normal AE, with nnMET this time. retry same settings from trial 125. Change MET such that MET=0 still remains 0."
}

#datasets, data_info = ef.load_and_preprocess(**data_info)

In [5]:
#training_info, data_info = ef.train_multiple_models(datasets, data_info, **training_info)
#del datasets

Booting up... initializing trainings of 10 models

starting training model 0...
127521/127521 ━━━━━━━━━━━━━━━━━━━━ 106s 825us/step
22504/22504 ━━━━━━━━━━━━━━━━━━━━ 20s 889us/step

Epoch 12: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 19: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 26: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.

Epoch 32: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.

Epoch 38: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-08.
model 0 success

starting training model 1...
127521/127521 ━━━━━━━━━━━━━━━━━━━━ 110s 857us/step
22504/22504 ━━━━━━━━━━━━━━━━━━━━ 20s 883us/step

Epoch 15: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
model 1 success

starting training model 2...
127521/127521 ━━━━━━━━━━━━━━━━━━━━ 110s 857us/step
22504/22504 ━━━━━━━━━━━━━━━━━━━━ 20s 867us/step

Epoch 15: ReduceLROnPlateau reducing learning rate to 0.0

2025-03-11 08:21:27.377736: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-11 08:21:27.446952: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-11 08:21:27.447355: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
# datasets, l1Seeded_results, l1All_results = ef.process_multiple_models(
#     training_info=training_info,
#     data_info=data_info,
#     plots_path=training_info['save_path']+'/plots',
#     target_rate=target_rate,
#     L1AD_rate=L1AD_rate,
#     obj_type=training_info['obj_type']
# )

powering on... preparing to run evals
Loaded EB_473255 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_473255.h5
Loaded EB_475321 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_475321.h5
Loaded EB_482596 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_482596.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/HLT_noalg_eb_L1All.h5
Loaded mc23e_ChiPlusChiMinus100_99_0p3ns from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_ChiPlusChiMinus100_99_0p3ns.h5
Loaded mc23e_ChiPlusChiMinus500_40_10ns from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_ChiPlusChiMinus500_40_10ns.h5
Loaded mc23e_HAHMggf from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_HAHMgg

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


137150/137150 ━━━━━━━━━━━━━━━━━━━━ 228s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 674ms/step
167097/167097 ━━━━━━━━━━━━━━━━━━━━ 268s 2ms/step
model 0:
L1AD_pure_rate: 993.9470406630284
L1AD_total_rate: 11413.139244071017
L1AD_threshold: 7.1612677574157715
l1Seeded:::
HLTAD_pure_rate: 9.909385642276114
HLTAD_total_rate: 53.5871356799523
HLTAD_threshold: 9.224163055419922

ROC threshold for tag HLT_noalg_eb_L1All was given to be 9.224163055419922
Corresponding ROC FPR is 0.004695214395795092
ROC threshold for tag mc23e_ChiPlusChiMinus100_99_0p3ns was given to be 9.224163055419922
Corres

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/eos/home-i03/m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/ensembler_functions.py:1661: RuntimeWarning: divide by zero encountered in scalar divide
  return x / (1 - x)
/eos/home-i03/m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/ensembler_functions.py:1709: RuntimeWarning: invalid value encountered in scalar subtract
  uncertainties_up[tag] = func(eff + eff_unc_up) - func(eff)
/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/matplotlib/axes/_axes.py:3683: RuntimeWarning: invalid value encountered in scalar add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err


l1All:::
HLTAD_pure_rate: 9.964189967201603
HLTAD_total_rate: 26.057418079444812
HLTAD_threshold: 12.982267379760742

ROC threshold for tag HLT_noalg_eb_L1All was given to be 12.982267379760742
Corresponding ROC FPR is 0.00026057418079447316
ROC threshold for tag mc23e_ChiPlusChiMinus100_99_0p3ns was given to be 12.982267379760742
Corresponding ROC FPR is 0.00026057418079447316
ROC threshold for tag mc23e_ChiPlusChiMinus500_40_10ns was given to be 12.982267379760742
Corresponding ROC FPR is 0.00026057418079447316
ROC threshold for tag mc23e_HAHMggf was given to be 12.982267379760742
Corresponding ROC FPR is 0.00026057418079447316
ROC threshold for tag mc23e_HNLeemu was given to be 12.982267379760742
Corresponding ROC FPR is 0.00026057418079447316
ROC threshold for tag mc23e_RS_G_hh_bbbb_AutoWidth_c10_M300 was given to be 12.982267379760742
Corresponding ROC FPR is 0.00026057418079447316
ROC threshold for tag mc23e_ZZ4lep was given to be 12.982267379760742
Corresponding ROC FPR is 0.000

...

l1All:::
HLTAD_pure_rate: 8.555313311277233
HLTAD_total_rate: 38.19180248917851
HLTAD_threshold: 10.75858211517334

ROC threshold for tag HLT_noalg_eb_L1All was given to be 10.75858211517334
Corresponding ROC FPR is 0.00038191802489182306
ROC threshold for tag mc23e_ChiPlusChiMinus100_99_0p3ns was given to be 10.75858211517334
Corresponding ROC FPR is 0.00038191802489182306
ROC threshold for tag mc23e_ChiPlusChiMinus500_40_10ns was given to be 10.75858211517334
Corresponding ROC FPR is 0.00038191802489182306
ROC threshold for tag mc23e_HAHMggf was given to be 10.75858211517334
Corresponding ROC FPR is 0.00038191802489182306
ROC threshold for tag mc23e_HNLeemu was given to be 10.75858211517334
Corresponding ROC FPR is 0.00038191802489182306
ROC threshold for tag mc23e_RS_G_hh_bbbb_AutoWidth_c10_M300 was given to be 10.75858211517334
Corresponding ROC FPR is 0.00038191802489182306
ROC threshold for tag mc23e_ZZ4lep was given to be 10.75858211517334
Corresponding ROC FPR is 0.000381918024

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/eos/home-i03/m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/ensembler_functions.py:1661: RuntimeWarning: divide by zero encountered in scalar divide
  return x / (1 - x)
/eos/home-i03/m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/ensembler_functions.py:1709: RuntimeWarning: invalid value encountered in scalar subtract
  uncertainties_up[tag] = func(eff + eff_unc_up) - func(eff)
/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/matplotlib/axes/_axes.py:3683: RuntimeWarning: invalid value encountered in scalar add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err


phase 1: starting evals of model 7...


/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


137150/137150 ━━━━━━━━━━━━━━━━━━━━ 219s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
167097/167097 ━━━━━━━━━━━━━━━━━━━━ 268s 2ms/step


In [5]:
# datasets, region_counts = ef.load_and_inference(training_info, data_info, tags='all', save_version=4)
# ef.save_subdicts_to_h5(datasets, '/eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025')

Loaded EB_473255 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-12-2025/EB_473255.h5
Loaded EB_475321 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-12-2025/EB_475321.h5
Loaded EB_482596 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-12-2025/EB_482596.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-12-2025/HLT_noalg_eb_L1All.h5
Loaded ZB_474448 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-12-2025/ZB_474448.h5
Loaded ZB_474462 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-12-2025/ZB_474462.h5
Loaded ZB_474562 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-12-2025/ZB_474562.h5
Loaded ZB_474657 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-12-2025/ZB_474657.h5
Loaded ZB_4746

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


l1Seeded::: target rate: 5
HLTAD_pure_rate: 4.956784325757746
HLTAD_total_rate: 29.369010107360676
HLTAD_threshold: 15.02770709991455

l1Seeded::: target rate: 20
HLTAD_pure_rate: 19.946187730454646
HLTAD_total_rate: 86.82247251625303
HLTAD_threshold: 8.768178939819336

l1Seeded::: target rate: 10
HLTAD_pure_rate: 9.894838723278164
HLTAD_total_rate: 57.190831580874914
HLTAD_threshold: 10.574394226074219

Saved HLT_noalg_eb_L1All to /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/HLT_noalg_eb_L1All.h5
Saved ZB_474448 to /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/ZB_474448.h5
Saved ZB_474462 to /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/ZB_474462.h5
Saved ZB_474562 to /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/ZB_474562.h5
Saved ZB_474657 to /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-20

2025-03-13 21:01:21.532922: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-13 21:01:21.796711: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-13 21:01:21.797065: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
# # # Convert to ONNX
# ef.convert_to_onnx(
#     training_info=training_info, 
#     data_info=data_info, 
#     model_version=4, 
#     save_dir='/eos/home-m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/onnx_models', 
#     opset=13, 
#     input_dim=48
# )

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_1 (Functional)       │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_9 (Functional)       │ (None)                 │        49,780 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,780 (194.45 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 49,780 (194.45 KB)

AttributeError: 'list' object has no attribute 'items'

2025-03-11 18:32:22.390060: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-11 18:32:22.470764: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-11 18:32:22.471142: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
# cpu
datasets, data_info = ef.load_and_preprocess(**data_info)
datasets, comparison_stats = ef.compare_tf_with_onnx(
    datasets, 
    training_info, 
    data_info, 
    model_version=4, 
    onnx_path='/eos/home-m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/onnx_models',
    tags=['EB_test', 'kaito']
)

Loaded EB_473255 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_473255.h5
Loaded EB_475321 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_475321.h5
Loaded EB_482596 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_482596.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/HLT_noalg_eb_L1All.h5
Loaded mc23e_ChiPlusChiMinus100_99_0p3ns from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_ChiPlusChiMinus100_99_0p3ns.h5
Loaded mc23e_ChiPlusChiMinus500_40_10ns from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_ChiPlusChiMinus500_40_10ns.h5
Loaded mc23e_HAHMggf from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_HAHMggf.h5
Loaded mc23e_HNLeemu from /eos/ho

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_swan/x86_64-el9-gcc13-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Comparison for EB_test:
Max absolute difference: 0.0
Mean absolute difference: 0.0
Number of significant differences: 0 out of 1336776

Sample of largest differences:
Index 1336772: TF=0.0, ONNX=0.0, Diff=0.0
Index 1336773: TF=3.607471466064453, ONNX=3.607471466064453, Diff=0.0
Index 1336774: TF=5.6854424476623535, ONNX=5.6854424476623535, Diff=0.0
Index 1336775: TF=3.800938606262207, ONNX=3.800938606262207, Diff=0.0
Index 0: TF=0.0, ONNX=0.0, Diff=0.0

Comparison for kaito:
Max absolute difference: 0.0
Mean absolute difference: 0.0
Number of significant differences: 0 out of 3

Sample of largest differences:
Index 0: TF=2.0157957077026367, ONNX=2.0157957077026367, Diff=0.0
Index 1: TF=5.69643497467041, ONNX=5.69643497467041, Diff=0.0
Index 2: TF=0.0, ONNX=0.0, Diff=0.0


In [5]:
# gpu
datasets, data_info = ef.load_and_preprocess(**data_info)
datasets, comparison_stats = ef.compare_tf_with_onnx(
    datasets, 
    training_info, 
    data_info, 
    model_version=4, 
    onnx_path='/eos/home-m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/onnx_models',
    tags=['EB_test', 'kaito']
)

Loaded EB_473255 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_473255.h5
Loaded EB_475321 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_475321.h5
Loaded EB_482596 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_482596.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/HLT_noalg_eb_L1All.h5
Loaded mc23e_ChiPlusChiMinus100_99_0p3ns from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_ChiPlusChiMinus100_99_0p3ns.h5
Loaded mc23e_ChiPlusChiMinus500_40_10ns from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_ChiPlusChiMinus500_40_10ns.h5
Loaded mc23e_HAHMggf from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_HAHMggf.h5
Loaded mc23e_HNLeemu from /eos/ho

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Comparison for EB_test:
Max absolute difference: 13.350914001464844
Mean absolute difference: 0.043570470064878464
Number of significant differences: 142664 out of 1336776

Sample of largest differences:
Index 1157609: TF=66.63313293457031, ONNX=58.30398941040039, Diff=8.329143524169922
Index 1105019: TF=283.6354675292969, ONNX=274.7718811035156, Diff=8.86358642578125
Index 972813: TF=53.398681640625, ONNX=44.4989013671875, Diff=8.8997802734375
Index 905050: TF=72.0141830444336, ONNX=63.01240921020508, Diff=9.001773834228516
Index 1217020: TF=80.10557556152344, ONNX=66.7546615600586, Diff=13.350914001464844

Comparison for kaito:
Max absolute difference: 0.20157980918884277
Mean absolute difference: 0.06719326972961426
Number of significant differences: 1 out of 3

Sample of largest differences:
Index 1: TF=5.69643497467041, ONNX=5.69643497467041, Diff=0.0
Index 2: TF=0.0, ONNX=0.0, Diff=0.0
Index 0: TF=2.2173755168914795, ONNX=2.0157957077026367, Diff=0.20157980918884277


2025-03-11 18:34:41.289316: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-11 18:34:41.357946: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-11 18:34:41.358260: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
idx=6534
print(datasets['EB_test']['HLT_calculated_AD_scores'][idx:idx+5])
print(datasets['EB_test']['HLT_AD_scores'][idx:idx+5])
print(datasets['EB_test']['ONNX_HLT_AD_scores'][idx:idx+5])
print('-'*50)
print(datasets['kaito']['HLT_calculated_AD_scores'])
print(datasets['kaito']['HLT_AD_scores'])
print(datasets['kaito']['ONNX_HLT_AD_scores'])

KeyError: 'HLT_calculated_AD_scores'

In [14]:
print(datasets['EB_test']['HLT_data'][10326]) # mismatch

[ 5.14656982e+01  2.88852620e+00  1.03940427e+00  3.63131714e+01
 -4.41292715e+00  1.92049825e+00  3.58155251e+01 -4.15592575e+00
 -1.03218806e+00  3.33892288e+01  4.11864281e-01 -2.35033846e+00
  3.19702339e+01 -3.08504581e+00  1.24078000e+00  3.01417351e+01
  4.12992191e+00 -2.65799522e+00  1.00809231e+01  4.90412921e-01
 -2.42034054e+00  6.02697229e+00  2.03080282e-01 -1.92615557e+00
  3.32211661e+00  3.09368312e-01 -1.75961185e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  9.28999424e+00  5.25033772e-01 -2.38488007e+00  9.28999424e+00
  5.25033772e-01 -2.38488007e+00  5.66200304e+00  1.49054006e-01
 -2.02224207e+00 -9.99000000e+02  0.00000000e+00 -9.99000000e+02]


In [17]:
print(datasets['EB_test']['HLT_data'][6536]) # mismatch

[67.7169342   2.59624243 -3.10504675 56.96654892 -1.52551913 -1.57162309
 51.41143036 -0.59155405  1.01663888 36.69818497 -3.15054989 -1.57943499
 29.48464012 -3.01455522  1.10164154 27.26969147  1.49540889  2.04630399
 20.40127754 -0.56405205  1.03506923 20.40127754 -0.56405205  1.03506923
 14.94297123  1.82882905  1.0525049   0.          0.          0.
  0.          0.          0.          0.          0.          0.
 20.04959679 -0.58002049  1.00969374 20.04959679 -0.58002049  1.00969374
 20.04959679 -0.58002049  1.00969374 44.38955307  0.          0.        ]


In [15]:
print(datasets['EB_test']['HLT_data'][10329]) # match

[65.49784088  2.65319777  0.38026142 33.51157761  3.07127929 -0.5016135
 33.23770905  1.32811236 -0.79255944 28.30817986 -3.96722865  3.10961485
 26.13201904  4.56805468 -0.75646836 23.87494278  4.27045155 -0.23124863
  4.98517656 -0.54066277  2.57884312  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  5.24832487  0.6465227  -2.36049986  4.86228228  0.94521648  0.62852472
  4.74037027  2.17469406 -0.3073692  54.76490402  0.          0.        ]


In [18]:
print(datasets['EB_test']['HLT_data'][6534]) # match

[ 2.94860802e+01  4.41474962e+00 -2.09029126e+00  2.65178814e+01
 -4.06527996e+00 -1.88999844e+00  2.62402477e+01  4.17639065e+00
 -2.90080738e+00  1.99991493e+01 -5.34521222e-01 -8.22378814e-01
  1.82962303e+01  2.83986020e+00 -1.21085274e+00  1.77394981e+01
 -3.19845051e-01 -3.02274084e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7.11596298e+00  1.43017673e+00  2.45163059e+00  3.94988751e+00
 -2.35258770e+00 -2.67913914e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -9.99000000e+02  0.00000000e+00 -9.99000000e+02]


### analyze the saved data

In [4]:
datasets = ef.load_subdicts_from_h5('/eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025')
ef.process_single_model_from_loaded_datasets(
    datasets=datasets,
    plots_path='/eos/home-m/mmcohen/ad_trigger_development/trained_models/trial_136/plots2',
    model_version=4,
    L1AD_rate=L1AD_rate
)

Loaded EB_test from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/EB_test.h5
Loaded EB_train from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/EB_train.h5
Loaded EB_val from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/EB_val.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/HLT_noalg_eb_L1All.h5
Loaded ZB_474448 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/ZB_474448.h5
Loaded ZB_474462 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/ZB_474462.h5
Loaded ZB_474562 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/ZB_474562.h5
Loaded ZB_474657 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_inferenced_data/03-12-2025/ZB_474657.h5
Loaded ZB_474679 from /eos

/eos/home-i03/m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/ensembler_functions.py:1677: RuntimeWarning: divide by zero encountered in scalar divide
  return x / (1 - x)
/eos/home-i03/m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/ensembler_functions.py:1725: RuntimeWarning: invalid value encountered in scalar subtract
  uncertainties_up[tag] = func(eff + eff_unc_up) - func(eff)
/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/matplotlib/axes/_axes.py:3683: RuntimeWarning: invalid value encountered in scalar add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



EB_test:
HLT_AD_scores: [0.        0.        0.        ... 3.6074715 5.685442  3.9366858]
HLT_data: [[ 3.15354290e+01 -3.38376498e+00 -2.77835155e+00 ... -9.99000000e+02
   0.00000000e+00 -9.99000000e+02]
 [ 3.05904694e+01  3.33570647e+00  2.62227249e+00 ... -9.99000000e+02
   0.00000000e+00 -9.99000000e+02]
 [ 2.63112411e+01 -4.37047482e+00  2.47191715e+00 ... -9.99000000e+02
   0.00000000e+00 -9.99000000e+02]
 ...
 [ 9.47099152e+01 -1.73136711e+00 -5.58597982e-01 ... -9.99000000e+02
   0.00000000e+00 -9.99000000e+02]
 [ 3.39098877e+02  1.03498316e+00  1.85774374e+00 ...  4.57073784e+01
   0.00000000e+00  0.00000000e+00]
 [ 4.40490143e+02  1.15914083e+00 -2.32770014e+00 ...  7.16322403e+01
   0.00000000e+00  0.00000000e+00]]
HLT_preprocessed_data: [[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 ...
 [ 4.735496  -1.7

/eos/home-i03/m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/ensembler_functions.py:2151: RuntimeWarning: invalid value encountered in scalar divide
  scale = 0.3 + 0.7 * np.log10(1 + pt)/np.log10(1 + max_electron_pt)
/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/numpy/lib/histograms.py:885: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 6 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 33 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph po

In [8]:
zero_mask = np.all(datasets['EB_test']['HLT_preprocessed_data'] == 0, axis=1)
latent_reps = datasets['EB_test']['z'][~zero_mask]
data = datasets['EB_test']['HLT_data'][~zero_mask]

print(latent_reps[:20])
print(data[1])

[[0.9962127  0.26555997 0.9669742  1.0076503 ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [3.9265714  0.26556003 3.6599154  4.9591017 ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [3.3538427  0.26556003 3.1335893  4.186805  ]
 [1.8467106  0.26556    1.748566   2.1545    ]
 [5.12382    0.26556006 4.760164   6.573531  ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [5.926826   0.26556006 5.4981117  7.656332  ]
 [3.4894066  0.26556003 3.2581716  4.3695955 ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [1.1406711  0.26555997 1.0997285  1.2024462 ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [1.121465   0.26555997 1.0820783  1.1765475 ]
 [0.9962127  0.26555997 0.9669742  1.0076503 ]
 [2.0934706  0.26556    1.9753318  2.4872508 ]]
[ 6.27044182e+01 -2.35620975e-01  2.36364222e+00  3.5450557

### Check the events kaito sent me

In [4]:
training_info = {
    "save_path": "/eos/home-m/mmcohen/ad_trigger_development/trained_models/trial_132", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": False, 
    "num_trainings": 10, 
    "training_weights": True, 
    "obj_type": "HLT", 
    "overlap_removal": False,
    "duplicate_removal": False
}

data_info = {
    "train_data_scheme": "topo2A_train", 
    "pt_normalization_type": "global_division", 
    "L1AD_rate": 1000, 
    "pt_thresholds": [50, 30, 30, 30], 
    "pt_scale_factor": 0.05, 
    "comments": "back to normal AE, with nnMET this time. retry same settings from trial 125"
}

datasets, data_info = ef.load_and_preprocess(**data_info)

# ef.convert_to_onnx(
#     training_info=training_info, 
#     data_info=data_info, 
#     model_version=5, 
#     save_dir='/eos/home-m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/onnx_models', 
#     opset=13, 
#     input_dim=48
# )

datasets = ef.compare_tf_with_onnx(
    datasets, 
    training_info, 
    data_info, 
    model_version=5, 
    onnx_path='/eos/home-m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/onnx_models'
)

Loaded EB_473255 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_473255.h5
Loaded EB_475321 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_475321.h5
Loaded EB_482596 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/EB_482596.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/HLT_noalg_eb_L1All.h5
Loaded mc23e_ChiPlusChiMinus100_99_0p3ns from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_ChiPlusChiMinus100_99_0p3ns.h5
Loaded mc23e_ChiPlusChiMinus500_40_10ns from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_ChiPlusChiMinus500_40_10ns.h5
Loaded mc23e_HAHMggf from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/03-03-2025/mc23e_HAHMggf.h5
Loaded mc23e_HNLeemu from /eos/ho

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Input data for kaito, shape: (3, 48)
Input data values: [[ 1.30671e+02 -6.76000e-01  1.13900e+00  6.44010e+01  2.07700e+00
  -1.51900e+00  4.24210e+01  1.02800e+00 -2.13200e+00  2.63810e+01
   5.12000e-01 -2.69800e+00  2.55680e+01  4.80000e-02  4.44000e-01
   0.00000e+00  0.00000e+00  0.00000e+00  7.32360e+01 -6.66000e-01
   1.11200e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   5.02660e+01  0.00000e+00 -2.70400e+00]
 [ 2.19340e+01 -6.44000e-01 -2.80700e+00  2.12680e+01  2.11800e+00
   1.71300e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.

2025-03-11 05:30:58.216865: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-11 05:30:58.292109: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-11 05:30:58.292462: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
# gpu
print(datasets['kaito']['ONNX_HLT_AD_scores'])
print(datasets['kaito']['HLT_AD_scores'])
print(datasets['kaito']['HLT_calculated_AD_scores'])

[2.0084577 5.747313  6.5237036]
[2.2093034 5.747313  6.5237036]
[2.20930347 5.74731302 6.52370358]


In [5]:
# cpu
print(datasets['kaito']['ONNX_HLT_AD_scores'])
print(datasets['kaito']['HLT_AD_scores'])
print(datasets['kaito']['HLT_calculated_AD_scores'])

[2.0084577 5.747313  6.5237036]
[2.0084577 5.747313  6.5237036]
[2.00845753 5.74731302 6.52370358]


In [7]:
print(datasets['kaito']['HLT_data'])

[[ 1.30671e+02 -6.76000e-01  1.13900e+00  6.44010e+01  2.07700e+00
  -1.51900e+00  4.24210e+01  1.02800e+00 -2.13200e+00  2.63810e+01
   5.12000e-01 -2.69800e+00  2.55680e+01  4.80000e-02  4.44000e-01
   0.00000e+00  0.00000e+00  0.00000e+00  7.32360e+01 -6.66000e-01
   1.11200e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   5.02660e+01  0.00000e+00 -2.70400e+00]
 [ 2.19340e+01 -6.44000e-01 -2.80700e+00  2.12680e+01  2.11800e+00
   1.71300e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00

In [8]:
# cpu
import tensorflow as tf
print(tf.__version__)

2.16.1


In [6]:
# gpu
import tensorflow as tf
print(tf.__version__)

2.16.1
